In [1]:
from itertools import combinations
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, collect_list, col
import networkx as nx

import matplotlib.pyplot as plt

In [2]:
PATH_TO_DATASETS = 'file:///home/wyvern/ING2/HACKATHON-2025/datasets/'

In [3]:
spark = SparkSession.builder \
    .appName("GenreCooccurrenceGraph") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/07/07 12:24:25 WARN Utils: Your hostname, Bhava-Agra resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/07/07 12:24:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/07 12:24:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
titles_df = spark.read.csv(PATH_TO_DATASETS + 'title.basics.tsv', sep='\t', header=True, nullValue='\\N')
exploded_genres_df = titles_df.withColumn("genre", explode(split(col("genres"), ',')))

In [5]:
genre_combinations = exploded_genres_df.groupBy('tconst').agg(collect_list('genre').alias('genres'))

In [6]:
def process_partition(partition):
    partition_graph = nx.Graph()
    for movie in partition:
        tconst, genres = movie
        if genres:
            for genre1, genre2 in combinations(genres, 2):
                if partition_graph.has_edge(genre1, genre2):
                    partition_graph[genre1][genre2]['weight'] += 1
                else:
                    partition_graph.add_edge(genre1, genre2, weight=1)
    return partition_graph

In [7]:
G = nx.Graph()

for partition in genre_combinations.rdd.mapPartitions(lambda x: [process_partition(x)]).collect():
    G = nx.compose(G, partition)

24/07/07 12:24:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=300, font_size=10, font_color='black', node_color='skyblue', edge_color='gray', linewidths=0.5)
plt.title('Genre Co-Occurrence Graph')
plt.show()

In [8]:
nx.write_graphml(G, '../graphs/genre_cooccurrence_graph.graphml')

In [9]:
spark.stop()